In [6]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.8.0-cp310-cp310-manylinux2010_x86_64.whl (497.6 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)


In [1]:
#The following script is supposed to predict next ten stock prices

import tensorflow as tf
import pandas as pd
import numpy as np

def xy_data(dataset):
    X_train=[]
    y_train=[]
    T = 5
    for t in range(len(dataset)-T):
        X_train.append(dataset[t:t+T])
        y_train.append(dataset[t+T])
    return X_train,y_train

def create_model(X_train,y_train):
    X_train = np.array(X_train).reshape(-1,5,1)
    y_train = np.array(y_train)
    # print(X_train.shape)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(5))
    model.add(tf.keras.layers.Dense(1,activation = 'relu'))
    model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(learning_rate=0.05))
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=20,restore_best_weights=True)
    n = list(X_train.shape)[0]
    model.fit(X_train[int(0.2*n):],y_train[int(0.2*n):],epochs=200,validation_data=(X_train[:int(0.2*n)],y_train[:int(0.2*n)]),steps_per_epoch=5,callbacks=[callback])
    return model
    
def predict(model,X):
    pred = []
    for _ in range(12):
        # print(X[-5:])
        data = np.array(X[-5:]).reshape(1,5,1)
        # print(data.shape)
        p = float(model.predict(data))
        X.append(p)
        pred.append(p)
    return pred
    
def get_pred(Ticker,dataset):
    ind = list(dataset['Ticker']).index(Ticker) 
    X_train,y_train = xy_data(list(dataset.transpose()[ind])[1:])
    model = create_model(X_train,y_train)
    predict_year = predict(model,list(dataset.transpose()[ind])[1:])
    return predict_year

def get_dict(preds):
    y=2022
    m=4
    dates = {}
    for i in range(12):
        dates[f'{y}-{m}-01'] = preds[i]
        if m==12:
            m = 1
            y +=1
        else:
            m+=1
    return dates
                                                             
stockdata = pd.read_csv("../data/stockdatawith0.csv")

try:
    preds = get_pred('adbe',stockdata)
except:
    preds=[]
    print('Ticker not found')

dict_pred = get_dict(preds)

2022-03-19 19:11:36.382100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-19 19:11:36.382121: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/200


2022-03-19 19:11:39.444342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-19 19:11:39.444609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-19 19:11:39.444696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-19 19:11:39.444766: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-19 19:11:39.444830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

5/5 [==============================] - 2s 134ms/step - loss: 179105.4688 - val_loss: 39753.5273
Epoch 2/200
5/5 [==============================] - 0s 12ms/step - loss: 178705.9219 - val_loss: 39558.0625
Epoch 3/200
5/5 [==============================] - 0s 17ms/step - loss: 178294.7656 - val_loss: 39364.4023
Epoch 4/200
5/5 [==============================] - 0s 15ms/step - loss: 177905.3594 - val_loss: 39169.3125
Epoch 5/200
5/5 [==============================] - 0s 17ms/step - loss: 177513.4531 - val_loss: 38974.4961
Epoch 6/200
5/5 [==============================] - 0s 18ms/step - loss: 177111.6875 - val_loss: 38781.9883
Epoch 7/200
5/5 [==============================] - 0s 17ms/step - loss: 176707.2031 - val_loss: 38591.1953
Epoch 8/200
5/5 [==============================] - 0s 17ms/step - loss: 176320.5625 - val_loss: 38399.0039
Epoch 9/200
5/5 [==============================] - 0s 18ms/step - loss: 175925.3594 - val_loss: 38207.4375
Epoch 10/200
5/5 [==============================

In [2]:
dict_pred

{'2022-4-01': 94.06629943847656,
 '2022-5-01': 94.06629943847656,
 '2022-6-01': 94.06629943847656,
 '2022-7-01': 94.06629943847656,
 '2022-8-01': 94.06629943847656,
 '2022-9-01': 94.06629943847656,
 '2022-10-01': 94.06629943847656,
 '2022-11-01': 94.06629943847656,
 '2022-12-01': 94.06629943847656,
 '2023-1-01': 94.06629943847656,
 '2023-2-01': 94.06629943847656,
 '2023-3-01': 94.06629943847656}